# Астероиды и кометы

## Заложим основы

In [ ]:
import numba as nb
import random
import numpy as np
import matplotlib.pyplot as plt
import unittest
from pathlib import Path
from PIL import Image
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
from IPython import display
from tqdm import tqdm
from dataclasses import dataclass

InteractiveShell.ast_node_interactivity = "all"

### Зададим объект
В моей программе только один класс, поскольку принципиальных различий между реальными небесными телами нет

In [ ]:
@dataclass
class CosmicBody:   
    existance: bool
    mass: float
    obj_radius: float
    radius_vec: list
    velocity_vec: list
    
    def dtChanges(self)


### Зададим методы
Их будет всего три:
- Рассчёт изменений координаты и скорости за dt
- Рассчёт столкновения (звучит гордо, но по факту не так)
- Определение типа траектории

In [ ]:
 def dtChanges(self)
 